<a href="https://colab.research.google.com/github/vpagonis/EURADOS-2023/blob/main/Code_15_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CODE 15.2 FITTING DOSE RESPONSE DATA USING 

Optical absorption (OA) and electron spin resonance (ESR)  signals are
often measured simultaneously with TL and/or OSL with the same sample.
These combined types of experiments produce important information
about the underlying luminescence mechanisms, as well as about the
nature of the trapping and luminescence centers (see for example the
quartz study by Yang and McKeever\index{Yang X and McKeever S W S!predose effect}
\cite{Yang_1990}, Chen and Pagonis \cite{Chen_2011_simulation}). 

The dose response of TL, OSL, OA and ESR signals exhibit nonlinear
regions, associated with the phenomena of superlinearity and sublinearity. 

A good example of simultaneous measurements and possible correlations
between the TL, ESR and OA signals\index{dose response!TL, ESR, OA signals}
has been found in a sample of fused silica (Wieser et al. \cite{Wieser1991}).
This type of multifaceted experiment can be very useful in identifying
the source of the various luminescence signals, i.e. the nature of
the traps and centers in the dosimetric material.

As discussed in the previous chapter, the most commonly used function
for fitting the dose response of luminescence signal is the saturating
exponential function (SE). Additional general dose response equations
are the saturating exponential plus linear function (SEL)\index{single saturating exponential plus linear SEL},
the double saturating exponential (DSE)\index{double saturating exponential DSE}
and the GOK-dose response (GOK-DR) equations. These equations can
be summarized as (Berger and Chen \cite{bergerchen2011}, Guralnik
et al. \cite{Guralnik_2015b}):
\begin{equation}
\frac{n(D)}{N}=1-\exp\left[-\frac{\,D}{D_{0}}\right]\:\qquad\textrm{SE}\label{eq:exE ch15}
\end{equation}
\begin{equation}
\frac{n(D)}{N}=B_{1}\:D+B_{2}\left(1-\exp\left[-\frac{\,D}{D_{0}}\right]\right)\:\qquad\textrm{SEL}\label{eq:SEL ch15}
\end{equation}
\begin{equation}
\frac{n(D)}{N}=B_{3}\left(1-\exp\left[-\frac{\,D}{D_{02}}\right]\right)+B_{4}\left(1-\exp\left[-\frac{\,D}{D_{01}}\right]\right)\:\quad\textrm{DSE}\label{eq:DSE ch15-1}
\end{equation}
\begin{equation}
\frac{n(D)}{N}=\left[1-\left(1+B\:b\:D\right)^{-1/b}\right]\qquad\textrm{GOK-DR}\label{eq:GOKDR-ch15}
\end{equation}
where $D$ is the irradiation dose, and $D_{0}$ is called the characteristic
dose of the trap filling process. $B_{i}$ $(i=1\ldots4)$ are constants,
and $D_{01},\:D_{02}$ are two constants characteristic of the sample
with the dimensions of dose $D$. As a rule of thumb when using the
SE function to fit data, the dose response is usually fully saturated
after a dose of about $5D_{0}$. In the case of the GOK-DR model,
$b$ is the GOK parameter and $A$, $B$ are constants.

Pagonis et al. \cite{Pagonis2020a} analyzed the relationship between
the PKC Eq.(\ref{eq:PKC ch15}) and the saturating exponential Eq.(\ref{eq:SE ch15}).
When $R\cong1$, the PKC Eq.(\ref{eq:PKC ch15}) becomes identical
with the SE in Eq.(\ref{eq:SE ch15}), while for small or large values
of $R$, (e.g. when $R=0.001$ or $R=3)$, there is disagreement between
the two expressions. 

These authors chose experimental data from the literature, which could
not be fitted with a SE function, and required instead the use of
a SEL or a DSE fitting function. They found that Eq.(\ref{eq:PKC ch15})
provides a satisfactory alternative to the empirical SEL and DSE regression
models. 


